In [78]:
# -*- coding: UTF-8 -*-
import time
import jieba


In [79]:
# jieba setting
relativePath = os.getcwd()
jieba.set_dictionary(relativePath + '/jieba_setting/dict.txt.big')
# add user dictionary to improve jieba cut precision
# jieba.load_userdict(relativePath + '/jieba_setting/yourfile.txt')

# stopwords setting
stopWordsSet = set()
with open(relativePath + '/jieba_setting/stopwords.txt', 'r') as stop:
    for i in stop:
        stopWordsSet.add(i.strip('\n'))

In [80]:
# 精確模式 ：將句子最精確地切開，叫適合文本分析, cut_all=False
# 全模式：把句子中所有的可以成詞的詞語都掃描出來, 速度快, cut_all=True
# 搜索引擎模式：在精確模式的基礎上對長詞再次切分，提高召回率，適合用於搜尋引擎分詞, jieba.cut_for_search(Content)            
# call jieba api
def jiebaCut(s):
    words = jieba.cut(s, cut_all=False)
    result = removeStopWords(words)
    return result

# remove stopwords
def removeStopWords(words):
    result = []
    for w in words:
        if w not in stopWordsSet:
            result.append(w)
    return result

# define all state 
def state(s,flag):
    nextline = 1
    if s is None or s == "":
        return flag, nextline
    # state: 1, s[0] = C
    if s[0] == 'C':
        flag, nextline = 1, 0
    # state: 2, s[0] = Q
    elif s[0] == 'Q':
        flag, nextline = 2, 0
    # state: 3, s[0] = A
    elif s[0] == 'A':
        flag, nextline = 3, 0
    # state: 4, do jieba cut
    return flag, nextline

In [81]:
import pickle
def outputFile(cList, qList, aList, ans):
    with open('CQA_Result.txt', 'a') as CQA:
        CQA.writelines('\ncorpus:\n')
        CQA.writelines(str(cList))
        CQA.writelines('\nquestion:\n')
        CQA.writelines(str(qList))
        CQA.writelines('\nanswer:\n')
        CQA.writelines(str(aList))
        CQA.writelines('\ncorrect_answer:\n')
        CQA.writelines(str(ans))

In [82]:

sTime = time.time()
print("Start process CQA dataset")
cNum = 0
with open('CQA.txt', 'r') as file:
    flag, end = 0, 0
    cList, qList, aList = [],[],[]
    ans = ""
    for i in file.readlines():
        s = i.strip()
        flag, nextline = state(s,flag)
        # one corpus process done!
        if end == 4:
            cNum +=1
            ans = s
            outputFile(cList, qList, aList, ans)
            print("Corpus: %d" % cNum)
            print("corpus:\n",cList,'\nquestion:\n',qList,'\nanswer:\n',aList,'\ncorrect ans:\n',ans,'\n')
            cList, qList, aList = [],[],[]
            flag, end = 0, 0
            ans = ""
            continue
        # still on state
        if nextline != 1:
            continue
        # on state 1, process Corpus
        elif flag == 1:
            print(s)
            cutRes = jiebaCut(s)
            for c in cutRes:
                cList.append(c)
        # on state 2, process Question
        elif flag == 2:
            cutRes = jiebaCut(s)
            for c in cutRes:
                qList.append(c)
        # on state 3, process Answer
        elif flag == 3:
            end += 1
            # example: （B） 吃飯比讀書更為重要 
            tempS = ""
            skip = ['A','B','C','D','（',')']
            check = 0
            for j in s:
                if check == 3:
                    tempS += j
                else:
                    check += 1
            tempS = tempS.strip()
            cutRes = jiebaCut(tempS)
            tempL = []
            for c in cutRes:
                tempL.append(c)
            aList.append(tempL)
            
print("\nTotal corpus numbers: %d" % cNum)
print("Processing all CQA dataset corpus took %.2fs" % (time.time()- sTime))

Building prefix dict from /home/ethan/pythonwork/ipynotebook/CQA/jieba_setting/dict.txt.big ...
Loading model from cache /tmp/jieba.u10984c2df34a568c519dc7b6af44e230.cache


Start process CQA dataset
有一種鳥叫蜂鳥，是鳥類中最小的鳥，只有拇指大小。這種鳥嘴 型細長，專吃花蜜和花上的小昆蟲，產於南美洲，整日盤旋於仙人掌 等幾種花間，也許營養太單調，總也吃不大、飛不高。而有些人讀書 做學問的方式，和蜂鳥很相似。這些人愛讀書，卻不大會讀書，往往 成為書的奴僕，他們埋進書堆，一心只讀自己的書，任天塌下來也不 管。這些人埋頭苦讀，精神可嘉，本無可厚非，然而令人擔心的是鑽 進象牙塔出不來。讀書就像吃飯，不可偏食，印刷的書籍要讀，人生 這本大書更得讀。


Loading model cost 2.020 seconds.
Prefix dict has been built succesfully.


Corpus: 1
corpus:
 ['有', '一種', '鳥叫', '蜂鳥', '是', '鳥類', '中', '最小', '的', '鳥', '只有', '拇指', '大小', '這種', '鳥嘴', '型', '細長', '專吃', '花蜜', '和', '花', '上', '的', '小', '昆蟲', '產於', '南美洲', '整日', '盤旋', '於', '仙人掌', '等', '幾種', '花間', '也許', '營養', '太', '單調', '總', '也', '吃', '不大', '飛不高', '而', '有些', '人', '讀書', '做學問', '的', '方式', '和', '蜂鳥', '很', '相似', '這些', '人愛', '讀書', '卻', '不', '大會', '讀書', '往往', '成為', '書', '的', '奴僕', '他們', '埋進', '書堆', '一心', '只讀', '自己', '的', '書', '任', '天塌下來', '也', '不', '管', '這些', '人', '埋頭', '苦讀', '精神可嘉', '本', '無可厚非', '然而', '令人擔心', '的', '是', '鑽', '進', '象牙塔', '出不來', '讀書', '就', '像', '吃飯', '不可', '偏食', '印刷', '的', '書籍', '要讀', '人生', '這本', '大書', '更得讀'] 
question:
 ['作者', '用', '蜂鳥', '來', '比喻', '哪', '一種', '人'] 
answer:
 [['不', '讀書', '的', '人'], ['只知', '讀書', '的', '人'], ['書讀', '得', '少', '的', '人'], ['假裝', '讀書', '的', '人']] 
correct ans:
 B 

有一種鳥叫蜂鳥，是鳥類中最小的鳥，只有拇指大小。這種鳥嘴 型細長，專吃花蜜和花上的小昆蟲，產於南美洲，整日盤旋於仙人掌 等幾種花間，也許營養太單調，總也吃不大、飛不高。而有些人讀書 做學問的方式，和蜂鳥很相似。這些人愛讀書，卻不大會讀書，往往 成為書的奴僕，他們埋進書堆，一心只讀自己的書，任天塌下來也不 管。這些人埋頭苦讀，精神可嘉，本無可厚

Corpus: 23
corpus:
 ['2008', '年', '的', '斯德哥爾摩', '水資源', '獎', '頒給', '了', '提出', '「', '虛擬', '水', '」', '概念', '的', '東尼', '艾倫', '教授', '所謂', '的', '「', '虛擬', '水', '」', '是', '指', '食品', '或', '消費品', '在', '生產', '及', '銷售', '過程', '中', '所', '需要', '的', '用水量', '這', '概念', '指出', '人們', '不僅', '在', '飲用', '和', '淋浴', '時', '需要', '水', '在', '消費', '其他', '產品', '時', '也', '會', '消耗', '大量', '的', '水', '例如', '一杯', '咖啡', '在', '種植', '生產', '包裝', '和', '運輸', '的', '過程', '中', '得', '消耗', '140', '升', '的', '水', '這', '相當於', '一個', '英國', '人', '平均', '每天', '飲用', '和', '家庭', '生活', '的', '用水量', '「', '虛擬', '水', '」', '的', '概念', '進一步', '說明', '了', '「', '虛擬', '水', '」', '的', '交易', '模式', '這對', '全', '球', '的', '貿易', '政策', '及', '水資源', '利用', '產生', '了', '重大', '影響', '這項', '理論', '指出', '氣候', '潮', '濕', '地區', '的', '農業', '多半會', '利用', '廉價', '的', '自然', '降雨', '而', '非', '成本', '高昂', '的', '灌溉', '系', '統', '因此', '能', '大幅', '降低', '農業', '的', '生產成本', '這些', '地區', '可以', '通過', '農作物', '將', '水資源', '「', '出口', '」', '到', '較', '為', '炎熱', '乾旱', '的', '國家', '而', '氣候', '向來', '乾旱',

...

Corpus: 263
corpus:
 ['現在', '請', '聽', '報導', '【', '新聞', '快訊', '】', '香港', '兩名', '到', '阿根廷', '遊玩', '的', '少女', '上週', '傳出', '在', '當地', '被', '人', '脅持', '禁錮', '及', '勒索', '贖金', '根據', '阿', '國', '警方', '調查', '她們', '可能', '涉及', '偷運', '古董', '阿', '國', '警方', '表示', '她們', '是', '為', '了', '獲得', '阿根廷', '免費', '旅遊', '及', '6', '萬元', '報酬', '才', '鋌而走險', '替人', '運送', '價值連城', '的', '古董', '花瓶', '和', '字畫', '到', '阿國', '但', '她們', '抵達', '阿根廷', '後', '接貨', '的', '幫派', '指稱', '花瓶', '破損', '於是', '扣留', '她們', '的', '護照', '加以', '禁錮', '香港', '警方', '接獲', '報案', '後', '動用', '超級', '電腦', '進行', '照片', '及', '阿根廷', '首都', '3D', '立體地圖', '的', '分析', '成功', '鎖定', '少女', '被', '禁錮', '的', '位置', '並將', '資料', '提供', '給阿國', '警方', '由', '他們', '負責', '救援', '行動', '阿', '國', '警方', '認為', '這次', '能', '成功', '地', '救出', '少女', '都', '要', '歸功於', '香港', '警方', '提供', '了極', '有用', '的', '情報'] 
question:
 ['根據', '報導', '兩名', '少女', '為', '什麼', '要', '做', '這些', '事'] 
answer:
 [['為', '了', '換取', '一頓', '溫飽'], ['為', '了', '一趟', '免費', '旅行'], ['為', '了', '價值連城', '的', '字畫'], ['為', '了', '救出', '被

Corpus: 289
corpus:
 ['「', '基因治療', '」', '是', '指將', '正常', '健康', '的', '基因', '移植', '到', '病患', '有', '缺失', '或是', '功能', '不全', '的', '基因', '中', '藉此', '治癒', '癌症', '或', '遺傳疾病', '等', '重大', '疾病', '以下', '是', '一段', '關於', '基因治療', '的', '討論', '請', '聽', '完', '後', '回答', '四個', '問題', '現在', '請', '聽', '對話', '男', '跟', '妳', '說', '個', '振奮人心', '的', '消息', '新', '的', '基因', '療法', '正', '準備', '進入', '臨床', '試驗', '哎', '等', '了', '這麼', '多年', '歷經', '了', '多少', '的', '研究', '基因治療', '總算', '要成', '為', '革命性', '的', '醫學', '療法', '了', '女', '嗯', '…', '這個', '消息', '對', '人類', '而言', '算不算', '是則', '福音', '還有', '待', '商榷', '吧', '畢竟', '治療', '後', '的', '基因', '在', '演化', '上會', '起', '什麼', '作用', '無人', '可知', '貝格', '博士', '就', '說', '了', '基因治療', '的', '同時', '也', '可能', '剔除', '掉', '一些', '對', '人類', '而言', '重要', '的', '基因', '像是', '鐮刀', '型', '貧血', '的', '基因', '缺陷', '其實', '對', '瘧疾', '就', '具有', '免疫力', '所以', '執行', '基因', '療法', '前', '實在', '得', '謹慎', '思考', '一旦', '改變', '了', '人類', '的', '基因', '會', '不會', '對', '未來', '世代', '造成', '其他', '傷害', '男', '你', '太', '杞人憂天', '了', '吧', '不論

Corpus: 323
corpus:
 ['男', '妳', '怎麼', '在家', '不', '和', '朋友', '到', '外面', '走走', '嗎', '女', '昨天', '玩得', '太累', '了', '今天', '又', '是', '寒假', '的', '最後', '一天', '當然', '要', '好好', '地', '在家', '休息', '啊'] 
question:
 ['男', '這位', '小姐', '為', '什麼', '在家'] 
answer:
 [['她', '覺得', '有點累'], ['她', '身體', '不', '舒服'], ['她', '剛剛', '才', '從', '外面', '回來'], ['她', '還沒想', '好', '去', '哪兒', '玩']] 
correct ans:
 A 

男：妳這麼喜歡出國旅遊，那妳多久旅遊一次？
女：差不多一年一次，每次去不同國家玩，去年去了韓國，今年打算去日本。
Corpus: 324
corpus:
 ['男', '妳', '這麼', '喜歡', '出國', '旅遊', '那', '妳', '多久', '旅遊', '一次', '女', '差不多', '一年', '一次', '每次', '去', '不同', '國家', '玩', '去年', '去', '了', '韓國', '今年', '打算', '去', '日本'] 
question:
 ['男', '這位', '小姐', '說', '了', '什麼'] 
answer:
 [['她', '一年', '出國', '旅遊', '兩次'], ['她', '今年', '不', '打算', '出國', '玩'], ['她', '每次', '去', '不', '一樣', '的', '國家', '玩'], ['她', '今年', '去', '韓國', '明年', '想', '去', '日本']] 
correct ans:
 C 

男：老師要我們寫的作業，妳寫得怎麼樣？
女：我想了好幾天，還是寫不出來。
Corpus: 325
corpus:
 ['男', '老師', '要', '我們', '寫', '的', '作業', '妳', '寫', '得', '怎麼樣', '女', '我', '想', '了', '好', '幾天', 

Corpus: 355
corpus:
 ['女', '國強', '我', '經常', '在', '台北市', '忠孝東路', '附近', '停車', '停', '一個', '小時', '要', '50', '元', '但', '如果', '我', '購物', '消費', '滿', '200', '元', '就', '可以', '免費', '停車', '一', '小時', '所以', '我常為', '了', '這個', '優惠', '花費', '更', '多', '男', '美珍', '這', '一點', '都', '不', '奇怪', '因為', '你', '掉', '進', '了', '「', '免費', '」', '的', '陷阱', '裡', '我舉', '個例', '子', '吧', '你', '知道', '那', '家', '知名', '的', '網路', '書店', '吧', '聽說', '早期', '他們', '所', '使用', '的', '策略', '跟', '現在', '不', '一樣', '那', '時候', '只要', '你', '的', '訂單', '金額', '達到', '…', '嗯', '假設', '是', '一千塊', '那麼', '運費', '其實', '都', '只要', '五十塊', '而已', '但', '即使', '運費', '是', '這麼', '地少', '它', '的', '吸引力', '還是', '不如', '免', '運費', '後來', '他們', '索性', '提高', '門檻', '改成', '購物滿', '兩千塊', '就', '可以', '免', '運費', '結果', '銷售量', '反而', '大大增加', '了', '你', '看', '其實', '你', '是', '花', '了', '更', '多', '的', '錢', '女', '有時候', '真', '搞不懂', '我們', '消費者', '的', '心態', '啊', '男', '其實', '在', '大部分', '的', '情況', '下', '拿', '免費', '的', '東西', '完全', '合理', '不拿白', '不', '拿', '嘛', '但是', '如果', '拿', '這個', '免費', '的', '東西', 


矮靈祭典是賽夏人最重要的祭典，舉辦的時間，大約是農曆十月，賽夏人稱這祭典為「巴斯答愛」。相傳在很久以前，有一群居住在山裡的矮人，身高只有三尺，但臂力很強，擅長法術，所以賽夏人很怕他們。
不過，由於矮人能歌善舞，所以賽夏人每年到了豐年祭，都會邀請矮人一同唱歌跳舞。只是，矮人卻趁這個機會侵犯賽夏族的婦女，在祭典過後，賽夏人才發現族裡許多婦女都懷孕了。因此，賽夏人越來越怨恨矮人。直到有一年的祭典，矮人又調戲賽夏族的婦女時，恰巧被賽夏人看見，賽夏人決定給矮人一個教訓，於是，他們暗中埋伏在矮人回去的路上，設下陷阱，讓矮人們一個一個跌落深淵而死，矮人傷亡慘重，只有兩人倖免於難。
這兩位矮人知道是自己咎由自取，便決定離開，離開前，還將祭歌與舞步教授給賽夏人。只是，賽夏人雖然除去了心頭大患，內心卻感到不安，於是開始祭祀矮人，安撫他們的靈魂，希望化解彼此的仇恨。
從此以後，就在秋收冬藏之際的月圓夜裡，賽夏人不斷地唱著、跳著，邀請矮靈歸來，再一次與賽夏人同樂，並在歌聲中請求矮靈的原諒與賜福。
Corpus: 385
corpus:
 ['矮靈', '祭典', '是', '賽夏人', '最', '重要', '的', '祭典', '舉辦', '的', '時間', '大約', '是', '農曆', '十月', '賽夏', '人稱', '這', '祭典', '為', '「', '巴斯', '答愛', '」', '相傳', '在', '很久以前', '有', '一群', '居住', '在', '山裡', '的', '矮', '人', '身高', '只有', '三尺', '但', '臂力', '很強', '擅長', '法術', '所以', '賽夏人', '很', '怕', '他們', '不過', '由於', '矮人', '能歌善舞', '所以', '賽夏人', '每年', '到', '了', '豐年祭', '都', '會', '邀請', '矮人', '一同', '唱歌跳舞', '只是', '矮人', '卻', '趁', '這個', '機會', '侵犯', '賽', '夏族', '的', '婦女', '在', '祭典', '過後', '賽夏', '人才', '發現', '族裡', '許多', '婦女', '都', '懷孕', '了', '因此', '賽夏人', '越來越', '怨恨', '矮人',